In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Dict, Tuple
import copy
import time
import numpy as np
from joblib import Parallel, delayed
from itertools import combinations, product
from queue import PriorityQueue, Empty
from helpers import *

In [19]:
nodes, costs, D = read_data(filename="TSPA.csv")

In [20]:
class Delta:
  def __init__(self, delta_value: int, delta_type: str, values: tuple):
      self.delta_value = delta_value
      self.delta_type = delta_type
      self.values = values
  def create_path(self, path):
      if self.delta_type == 'inter':
          index_of_old_node, new_value = self.values
          modified_path = copy.deepcopy(path)
          modified_path[index_of_old_node] = new_value
          return modified_path
      elif self.delta_type == 'edges_intra':
          index1, index2 = self.values
          if index2 == 0:
              modified_path = np.array(list(path[:index1+1]) + list(path[index2-1:index1:-1]))
          else:
              modified_path = np.array(list(path[:index1+1]) + list(path[index2-1:index1:-1]) + list(path[index2:]))
          return modified_path

def inter(path, distance_matrix, costs):
    og_nodes = set(path)
    nodes = set(list(range(distance_matrix.shape[0])))
    changes = product(og_nodes, nodes - og_nodes)
    node_indices = {node: index for index, node in enumerate(path)}
    neighbourhood_deltas = []
    for change in changes:
        old_node, new_node = change
        delta_value = 0
        index_of_old_node = node_indices[old_node]
        prev = path[(index_of_old_node - 1) % len(path)]
        next = path[(index_of_old_node + 1) % len(path)]
        old_dist = distance_matrix[prev, old_node] \
                  + distance_matrix[old_node, next]
        new_dist = distance_matrix[prev, new_node] \
                  + distance_matrix[new_node, next]
        delta_value += new_dist - old_dist
        delta_value += costs[new_node] - costs[old_node]
        delta = Delta(delta_value, 'inter', (index_of_old_node, new_node))
        neighbourhood_deltas.append(delta)
    return neighbourhood_deltas

def intra(path, distance_matrix):
    pairs1 = np.column_stack((path[:-1], path[1:]))
    cyclic_pair = np.array([path[-1], path[0]])
    all_edges = np.vstack((pairs1, cyclic_pair))
    edge_swaps = combinations(all_edges, 2)
    neighbourhood_deltas = []
    node_indices = {node: index for index, node in enumerate(path)}
    for swap in edge_swaps:
        edge1, edge2 = swap
        if edge1[1] == edge2[0] or edge1[0] == edge2[1]:
            continue
        delta_value = (
            distance_matrix[edge1[0], edge2[0]]
            + distance_matrix[edge1[1], edge2[1]]
        ) - (
            distance_matrix[edge1[0], edge1[1]]
            + distance_matrix[edge2[1], edge2[0]]
        )
        index1 = node_indices[edge1[0]]
        index2 = node_indices[edge2[1]]
        delta = Delta(delta_value, 'edges_intra', (index1, index2))
        neighbourhood_deltas.append(delta)
    return neighbourhood_deltas



def local_search(current_path, D, costs):
    while True:
        neighbourhood_deltas = inter(current_path, D, costs) + intra(current_path, D)
        min_index = np.argmin([delta.delta_value for delta in neighbourhood_deltas])
        best_delta = neighbourhood_deltas[min_index]
        if best_delta.delta_value < 0:
            current_path = best_delta.create_path(current_path)
        else:
            break
    return current_path


def multiple_start_local_search(D, costs, ls_iter=10):
    best_path = None
    best_score = MAX_DIST
    for i in range(ls_iter):
        initial_path = random_sequence(D)
        path = local_search(initial_path, D, costs)
        score = evaluate(D, path, costs)
        if score < best_score:
            best_path = path
            best_score = score
    return best_path


In [21]:
def evaluate_solution(filename, D, costs):
    start = time.time()
    path = multiple_start_local_search(D, costs)
    end = time.time()
    score = evaluate(D, path, costs)
    return {
        'Filename': filename,
        'Path': path,
        'Score': score,
        'Time': (end - start)
    }

In [22]:
result = evaluate_solution('TSPA.csv', D, costs)
result

{'Filename': 'TSPA.csv',
 'Path': array([ 98, 190,  72,  94,  89, 111,  14,  31,  73,  95, 169,  80, 124,
          8,  26, 119, 130,  48, 106,  11, 152, 189,  75,   1, 177,  41,
        199, 192,   4,  77,  43,  35,  19,  69,   0, 149,  50, 121,  91,
        114, 175, 153,  88, 127, 186,  79, 194,  21, 171, 117,  53,  22,
        195,  55,  36, 132, 128,  40,  34, 164, 178, 159, 143,  59, 147,
        116,  27,  96,  37, 185,  64, 181, 113,  61,  71,  20, 183, 103,
        163,  74,  62,  32, 180, 108,  81, 154, 144,  87, 141,  24,  45,
        167,  60, 101, 135,  51, 112,  66,   6, 156]),
 'Score': 76772,
 'Time': 59.6996169090271}

In [770]:
if __name__ == "__main__":
    test_start = time.time()
    files = ['TSPA.csv', 'TSPB.csv', 'TSPC.csv', 'TSPD.csv']
    for filename in files:
        prefix = "MSLS"
        mapping_out_files= {
            'TSPA.csv' : f'{prefix}_TSPA_out.csv',
            'TSPB.csv' : f'{prefix}_TSPB_out.csv',
            'TSPC.csv' : f'{prefix}_TSPC_out.csv',
            'TSPD.csv' : f'{prefix}_TSPD_out.csv'
        }
        nodes, costs, D = read_data(filename)
        results = Parallel(n_jobs=-1)(delayed(evaluate_solution)(filename, D, costs)
                                    for i in range(20)
                                    )

        results_df = pd.DataFrame(results)
        results_df.to_csv(mapping_out_files[filename])
        test_end = time.time()
        print(test_end - test_start)

TypeError: 'NoneType' object is not iterable